# Setup

In [ ]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from tqdm import tqdm_notebook
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score, cohen_kappa_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

# Import Data

In [ ]:
impt_data = pd.read_csv("CLC_train.csv")
test_data = pd.read_csv("CLC_test.csv")

In [ ]:
labels_pd = impt_data.pop("CO_level")
labels_test_pd = test_data.pop("CO_level")

In [ ]:
include_columns = ["CO_GT", "PT08_S1_CO", "PT08_S2_NMHC"]
data_pd = impt_data[include_columns]
data_test_pd = test_data[include_columns]

In [ ]:
data = data_pd.to_numpy()
labels = labels_pd.to_numpy()
X_test = data_test_pd.to_numpy()
Y_test = labels_test_pd.to_numpy()
print(data.shape)
print(X_test.shape)

# Train Test Split

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(data, labels, stratify = labels, random_state = 0, test_size = 0.4)

In [ ]:
enc = OneHotEncoder()
Y_OH_train = enc.fit_transform(np.expand_dims(Y_train, 1)).toarray()
Y_OH_val = enc.transform(np.expand_dims(Y_val, 1)).toarray()
Y_OH_test = enc.transform(np.expand_dims(Y_test, 1)).toarray()

In [ ]:
X_train, Y_OH_train, X_val, Y_OH_val, X_test, Y_OH_test = map(torch.tensor, (X_train, Y_OH_train, X_val, Y_OH_val, X_test, Y_OH_test))

In [ ]:
X_train, X_val, X_test = X_train.float(), X_val.float(), X_test.float()

# Cuda Support

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X_train = X_train.to(device)
X_val = X_val.to(device)
X_test = X_test.to(device)
Y_OH_train = Y_OH_train.to(device)
Y_OH_val = Y_OH_val.to(device)
Y_OH_test = Y_OH_test.to(device)

# FF Network

In [ ]:
class FF_Network(nn.Module):
    def __init__(self):
        super().__init__()
        torch.manual_seed(0)
        self.net = nn.Sequential(
            nn.Linear(3, 55),
            nn.BatchNorm1d(55),
            nn.ReLU(),
            nn.Linear(55,40),
            nn.BatchNorm1d(40),
            nn.ReLU(), 
            nn.Linear(40, 5),
        )
           
    def forward(self, X):
        return(self.net(X))

In [ ]:
ff_n = FF_Network()
ff_n = ff_n.to(device)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
def accuracy(Y_hat, Y):
    accuracy = accuracy_score(np.argmax(Y_hat.cpu().detach().numpy(), 1), np.argmax(Y.cpu().detach().numpy(), 1))
    f1_scr = f1_score(np.argmax(Y_hat.cpu().detach().numpy(), 1), np.argmax(Y.cpu().detach().numpy(), 1), average='macro')
    kappa = cohen_kappa_score(np.argmax(Y_hat.cpu().detach().numpy(), 1), np.argmax(Y.cpu().detach().numpy(), 1))
    return(accuracy, f1_scr, kappa)

In [ ]:
def train(X, Y, model, loss_fn = loss_fn, lr = 0.02, epochs = 2500, batch_size = 512):
    loss_b = []
    acc_b = []
    acc = 0
    optm = optim.Adam(ff_n.parameters(), lr = lr)
    Y_temp = Y.to('cpu')
    Y = np.argmax(Y_temp, 1)
    Y = Y.to(device)
    for i in tqdm_notebook(range(epochs)):
        for i in range(X.shape[0] // batch_size):
            local_X, local_Y, local_y_temp = X[i*batch_size:(i+1)*batch_size,], Y[i*batch_size:(i+1)*batch_size,], Y_temp[i*batch_size:(i+1)*batch_size,]
            Y_hat = model.forward(local_X)
            loss = loss_fn(Y_hat, local_Y)
            acc, _, _ = accuracy(Y_hat, local_y_temp)
            loss_b.append(loss.item())
            acc_b.append(acc)
            loss.backward()
            optm.step()
            optm.zero_grad()
    return(loss_b, acc_b, acc)

In [ ]:
%%time
loss_b, acc_b, acc_score = train(X_train, Y_OH_train, ff_n, loss_fn, lr = 0.02, epochs = 1500, batch_size = 2048)

In [ ]:
fig = plt.figure(figsize = (16, 8))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
ax1.plot(loss_b)
ax1.set_xlabel('time progress')
ax1.set_ylabel('loss')
ax2.plot(acc_b)
ax2.set_xlabel('time_progress')
ax2.set_ylabel('acc')

In [ ]:
# ff_n.eval()
print("Train set accuracy, f1 score: ", accuracy(ff_n.forward(X_train), Y_OH_train), "\nValidation set accuracy, f1 score: ", accuracy(ff_n.forward(X_val), Y_OH_val), "\nTest set accuracy, f1 score: ", accuracy(ff_n.forward(X_test), Y_OH_test))

# Visual Evaluation

In [ ]:
cm = confusion_matrix(np.argmax(Y_OH_val.to('cpu').detach().numpy(), 1), np.argmax(ff_n(X_val).to('cpu').detach().numpy(), 1))
print(cm)
plt.imshow(cm, cmap='binary')

In [ ]:
cm = confusion_matrix(np.argmax(Y_OH_train.to('cpu').detach().numpy(), 1), np.argmax(ff_n(X_train).to('cpu').detach().numpy(), 1))
print(cm)
plt.imshow(cm, cmap='binary')

In [ ]:
cm = confusion_matrix(np.argmax(Y_OH_test.to('cpu').detach().numpy(), 1), np.argmax(ff_n(X_test).to('cpu').detach().numpy(), 1))
print(cm)
plt.imshow(cm, cmap='binary')

# Saving Model

In [ ]:
# torch.save(ff_n.state_dict(), './weights_trials')

In [ ]:
model = FF_Network()
model.load_state_dict(torch.load('./weights'))
model.eval()
model.to(device)

In [ ]:
Y_test_pred = model.forward(X_test)

In [ ]:
print("Train set Accuracy, F1 score, Cohen's Kappa: ", accuracy(model.forward(X_train), Y_OH_train), "\nValidation set Accuracy, F1 score, Cohens Kappa: ", accuracy(model.forward(X_val), Y_OH_val), "\nTest set Accuracy, F1 score  Cohens Kappa: ", accuracy(model.forward(X_test), Y_OH_test))

In [ ]:
class_label = ["High", "Low", "Moderate", "Very High", "Very Low"]

In [ ]:
import csv

with open('submission_trials.csv', 'w', newline='') as file:
    with open('CLC_test.csv', 'r') as inp:
        writer = csv.writer(file)
        reader = csv.reader(inp)
        heading = next(reader)
        heading.append("Our Prediction")
        writer.writerow(['Date', 'Time', 'CO_GT', 'PT08_S1_CO', 'NMHC_GT', 'C6H6_GT', 'PT08_S2_NMHC', 'Nox_GT', 'PT08_S3_Nox', 'NO2_GT', 'PT08_S4_NO2', 'PT08_S5_O3', 'T', 'RH', 'AH', 'CO_level', 'Our prediction'])
        for i, row in enumerate(reader):
            row.append(class_label[np.argmax(Y_test_pred.to('cpu').detach().numpy(), 1)[i]])
            writer.writerow(row)